# Data Wrangling Report

## Introduction
This project was submitted as part of Udacity's Data Analyst Nanodegree program. Within this notebook data wrangling techniques are used to obtain data that is analyzed and visualized. The project motivation is to gain insights about the top dog breeds rated by [WeRateDogs](https://twitter.com/dog_rates9). This document briefly describes the data wrangling efforts.

## Gathering Data
The data will be gathered from 3 different data sources:
1. The WeRateDogs Twitter archive `data/twitter-archive-enhanced.csv` has been provided by Udacity. The file will be loaded into a dedicated DataFrame called `df_archive` using pandas.
1. The file `data/image_predictions.tsv` is hosted on Udacity's servers and will be downloaded programmatically using the Requests library. It contains the tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. The file will be loaded into a dedicated DataFrame called `df_img` using pandas.
1. Using the tweet IDs in the WeRateDogs Twitter archive, the Twitter API for each tweet's JSON data using Python's Tweepy library was queried. Each tweet's entire set of JSON data will be stored i a file called `data/tweet_json.txt file`. Each tweet's JSON data is stored to its own line. The tweet ID, retweet count, and favorite count will be loaded from this file into a dedicated DataFrame called `df_api` using pandas. To avoid exposing the Twitter developer accounts credentials used for accessing the Twitter API, the API Key was set as environment vairable `CONSUMER_KEY` and the secret as environment vairable `CONSUMER_SECRET`.

## Assessing Data
After gathering each of the above pieces of data, the data will be visually and programmatically assessed for quality and tidiness issues. The columns of each DataFrame will be described. The detected issues will be documented. Only issues that help to satisfy the project motivation will be covered.

**Following observations were made:**
* There are no relevant duplicates, no dropping of duplicates required.
* The column names are descriptive and do not include blanks. 

### Column description table `df_archive`
* `tweet_id` is the last part of the tweet URL after "status/", e.g. https://twitter.com/dog_rates/status/892420643555336193/
* `in_reply_to_status_id` will contain the integer representation of the original Tweet’s ID, if the represented Tweet is a reply
* `in_reply_to_user_id` will contain the integer representation of the original Tweet’s author ID, if the represented Tweet is a reply
* `timestamp` is the UTC time when this Tweet was created
* `source` is the Utility used to post the Tweet, as an HTML-formatted string
* `text` is the actual UTF-8 text of the status update
* `retweeted_status_id` will contain the integer representation of the original Tweet’s ID, if the represented Tweet is a retweet
* `retweeted_status_user_id` will contain the integer representation of the original Tweet’s author ID, if the represented Tweet is a retweet
* `retweeted_status_timestamp` will contain the UTC time when the original Tweet was created, if the represented Tweet is a retweet
* `expanded_urls` expanded version of the URL pasted/typed into Tweet
* `rating_numerator` the rating numerator defining the rating of the specifc dog associated with this tweet
* `rating_denominator` the rating denominator defining the rating of the specifc dog associated with this tweet
* `name` the name of the specifc dog associated with this tweet
* `doggo` is a dog stage
* `floofer` is a dog stage
* `pupper` is a dog stage
* `puppo` is a dog stage

### Column description table `df_img`
* `tweet_id` is the last part of the tweet URL after "status/", e.g. https://twitter.com/dog_rates/status/892420643555336193/
* `jpg_url` is the URL to the image
* `img_num` is the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images)
* `p1` is the algorithm's #1 prediction for the image in the tweet
* `p1_conf` is how confident the algorithm is in its #1 prediction
* `p1_dog` is whether or not the #1 prediction is a breed of dog
* `p2` is the algorithm's second most likely prediction
* `p2_conf` is how confident the algorithm is in its #2 prediction
* `p2_dog` is whether or not the #2 prediction is a breed of dog
* `p3` is the algorithm's second most likely prediction
* `p3_conf` is how confident the algorithm is in its #2 prediction
* `p3_dog` is whether or not the #2 prediction is a breed of dog

### Column description table `df_api`

* `id` is the last part of the tweet URL after "status/", e.g. https://twitter.com/dog_rates/status/892420643555336193/
* `favorite_count` indicates approximately how many times this Tweet has been liked by Twitter users
* `retweet_count` is the number of times the tweet has been retweeted

<a id='a-quality'></a>
### Quality Issues

#### `df_archive` table
* Some tweets are retweets and should be dropped, because they might be no original ratings
* Some tweets are replies and should be dropped, because they might be no original ratings (additionally, the columns `in_reply_to_status_id` and `in_reply_to_user_id` are `float64` instead of `int64` but this extraneous)
* The rating is sometimes incorrect by comparing text to `rating_numerator` and `rating_denominator` outliers
* The rating needs to be normalized by `rating_numerator` divided by `rating_denominator` due to the outliers
* The columns `name`, `doggo`, `floofer`, `pupper` and `puppo` contain the string `None` instead of `np.nan` as null value
* The column `name` contains very likely not real names, i.e. `a`, `by`, `unacceptable`, `very`, `an`, `my` and `incredibly`.
* The `timestamp` and `retweeted_status_timestamp` is represented as string and should be a datetime value (only `timestamp` requires cleaning, because retweets will not be considered)

#### `df_img` table
* If a dog was detected, then get the dog breed based on the highest confidence prediction
* The confidence of some predicted dog breeds is below 30%
* The predicted dog breed consisting of more than one word for p1 or p2 or p3 are separated by `_` or `-` instead of blank. Additionally, the words sometimes do not start capitalized 

#### `df_api` table
* No issues identified

<a id='a-tidiness'></a>
### Tidiness Issues
* In table `df_archive` each dog stage has it's own column (`doggo`, `floofer`, `pupper` and `puppo`), which will be changed to a single column `dog_stage`
* The table `df_archive` has tweets that are not represented in the table `df_img` and `df_api`
* The table `df_img` should be part of the `df_archive` table
* The table `df_api` should be part of the `df_archive` table

## Cleaning Data
There were no missing data issues to be cleaned first. That's why tidiness issues were cleaned first. Finally, quality issues were cleaned. The results of cleaning will be a high quality and tidy master pandas DataFrame `df_archive_clean` that is required to satisfy the Project Motivation. For each issue the cleaning process is structures by define, code, and test steps which are documented.

The results of cleaning is a high quality and tidy master pandas DataFrame `df_archive_clean` that is required to satisfy the Project Motivation. This master DataFrame will be saved as `data/twitter_archive_master.csv` using pandas. The DataFrame `df_api_cleaned` will be additionally to the file `data/tweet_json.txt` produced by querying Twitter's API stored as CSV file `data/twitter_api_cleaned.csv`. It's not required for further analysis, because the data was merged to the master DataFrame. Still, the CSV file makes it easier to access this data compared to the original produced txt-file. The DataFrame `df_img_clean` was not stored as CSV, because the data was merged to the master DataFrame.

The wrangled data provided within the master DataFrame `df_archive_clean` will be analyzed to produce insights and visualizations that satisfy the project motivation.